Unzipping Dataset

In [1]:
import pickle
import numpy as np
import pandas as pd
import os

directory = '../data/WESAD.zip'
output_directory = '../data/'

os.listdir(output_directory)

import zipfile
with zipfile.ZipFile(directory, 'r') as zip_ref:
    zip_ref.extractall(output_directory)

Plotting Individual Data

In [1]:
import pickle
import os
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from ecgdetectors import Detectors
from scipy import signal
from scipy.stats import skew,kurtosis,iqr
import pickle
from peak_valley import compute_peak_valley
from respiration_feature import rip_cycle_feature_computation
filelists = ['../data/WESAD/'+a+'/'+a+'.pkl' for a in os.listdir('../data/WESAD/') if a[-1] not in ['s','f']]

In [2]:
import warnings
warnings.filterwarnings('ignore')
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from scipy.signal import find_peaks
plt.close('all')
def filter_data(X,
                Fs=64,
                low_cutoff=.5,
                high_cutoff=3.0,
                filter_order=1):
    """
    Bandpass Filter of single channel

    :param X: input data
    :param Fs: sampling freq.
    :param low_cutoff: low passband
    :param high_cutoff: high passband
    :param filter_order: no of taps in FIR filter

    :return: filtered version of input data
    """
    X1 = X.reshape(-1,1)
    X1 = signal.detrend(X1,axis=0,type='constant')
    b = signal.firls(filter_order,np.array([0,low_cutoff-.1, low_cutoff, high_cutoff ,high_cutoff+.5,Fs/2]),np.array([0, 0 ,1 ,1 ,0, 0]),
                     np.array([100*0.02,0.02,0.02]),fs=Fs)
    X2 = signal.convolve(X1.reshape(-1),b,mode='same')
    return X2

class heart_rate:
    def __init__(self,lower_hr_range=.7,higher_hr_range=3.5,height=.2):
        self.hr_now = 0
        self.history_hr = []
        self.lower_hr_range = lower_hr_range
        self.higher_hr_range = higher_hr_range
        self.height = height
        self.previous = 0
        self.step = 2
    
    def filter_frequencies(self,x_x,f_x,pxx_x):
        x_x = np.array(x_x)
        x_x = x_x[f_x[x_x]>self.lower_hr_range]
        x_x = x_x[f_x[x_x]<self.higher_hr_range]
        f_x = f_x[x_x]
        pxx_x = pxx_x[x_x]
        return f_x,pxx_x
    
    def get_peaks(self,data,fs,nperseg,nfft):
        f_x, pxx_x = signal.welch(data,fs=fs,nperseg=nperseg,nfft=nfft)
        f_x = f_x.reshape(-1)
        pxx_x = pxx_x/np.max(pxx_x)
        x_x, _ = find_peaks(pxx_x, height=self.height)
        f,pxx = self.filter_frequencies(x_x,f_x,pxx_x)
        ppg = np.array(list(zip(f,pxx)))
        if len(ppg)==0:
            return []
        ppg = ppg[ppg[:,1].argsort()]
        return ppg
    
    def get_rr_value(self,values,acc_window,ecg_rr,Fs=64,nfft=12000):
        """
        Get Mean RR interval

        :param values: single channel ppg data
        :param Fs: sampling frequency
        :param nfft: FFT no. of points
        :return: Mean RR interval Information
        """
        values = filter_data(values)
        ppg = self.get_peaks(values,Fs,values.shape[0],nfft)[-3:]
        
        if len(ppg)==0:
            if len(self.history_hr)==0:
                return 0
            self.hr_now = np.mean(self.history_hr)
            self.history_hr.append(self.hr_now)
            self.history_hr  =self.history_hr[-6:]
            return 60000/self.hr_now
        
        aclx = list(self.get_peaks(acc_window[:,0],Fs//2,acc_window.shape[0],nfft)[-1:])
        acly = list(self.get_peaks(acc_window[:,1],Fs//2,acc_window.shape[0],nfft)[-1:])
        aclz = list(self.get_peaks(acc_window[:,2],Fs//2,acc_window.shape[0],nfft)[-1:])
        acl_unwanted = np.array(aclx+acly+aclz)
        if len(acl_unwanted)>0:
            acl_unwanted = acl_unwanted[:,0]*60
            hr_now = 0
            for k in range(1,len(ppg)+1):
                hr_temp = ppg[-1*k,0]*60
                if len(acl_unwanted[np.where((acl_unwanted>hr_temp-self.step)&(acl_unwanted<hr_temp+self.step))[0]])==0:
                    hr_now = hr_temp
                    break
        else:
            hr_now = ppg[-1,0]*60
        
        if hr_now==0:
            hr_now = ppg[-1,0]*60
        

        if not self.hr_now:
            self.hr_now = hr_now
            self.history_hr.append(self.hr_now)
            self.history_hr  =self.history_hr[-6:]
            return 60000/self.hr_now
        else:
            if abs(self.hr_now-hr_now)>15:
                if self.previous>5:
                    self.history_hr.append(hr_now)
                    self.hr_now = np.mean(self.history_hr)
                    self.history_hr.append(self.hr_now)
                    self.history_hr  =self.history_hr[-6:]
                    self.previous=1
                    return 60000/self.hr_now
                else:
                    self.hr_now = np.mean(self.history_hr)
                    self.history_hr.append(self.hr_now)
                    self.history_hr = self.history_hr[-6:]
                    self.previous+=1
                    return 60000/self.hr_now
                    
            else:
                self.hr_now = hr_now
                self.history_hr.append(self.hr_now)
                self.history_hr  =self.history_hr[-6:]
                return 60000/self.hr_now

In [3]:
from scipy.stats import mode
%matplotlib notebook
def get_ecg_rr(ecg_data):
    detectors = Detectors(700)
    rpeaks = detectors.hamilton_detector(ecg_data[:,1])
    ecg_r_ts = ecg_data[np.array(rpeaks),0]
    ecg_rr_ts = ecg_r_ts[1:]
    ecg_rr_sample = np.diff(ecg_r_ts)
    ecg_rr = pd.DataFrame(np.vstack([ecg_rr_ts,ecg_rr_sample]).T,columns=['time','rr'])
    ecg_rr['timestamp'] = ecg_rr['time'].apply(lambda a:datetime.utcfromtimestamp(a))
    return ecg_rr

def bandpass_filter_ppg(data,Fs=64,fil_type='ppg'):
    X0 = data[:,1]
    X1 = signal.detrend(X0,axis=0,type='constant')
    X2 = np.zeros((np.shape(X1)[0],data.shape[1]))
    nyq = Fs/2
    b = signal.firls(219,np.array([0,0.3,0.5,3,3.5,nyq]),
                              np.array([0,0,1,1,0,0]),np.array([10,1,1]),nyq=nyq)
    a = [1]
    X2[:,0] = data[:,0]
    X2[:,1] = signal.filtfilt(b, a, X1)
    return X2

def bandpass_filter_respiration(data,Fs=700,fil_type='ppg'):
    X0 = data[:,1]
    X1 = signal.detrend(X0,axis=0,type='constant')
    X2 = np.zeros((np.shape(X1)[0],data.shape[1]))
    nyq = Fs/2
    b = signal.firls(219,np.array([0,0.02,0.05,1,1.5,nyq]),
                              np.array([0,0,1,1,0,0]),np.array([10,1,1]),nyq=nyq)
    a = [1]
    X2[:,0] = data[:,0]
    X2[:,1] = signal.filtfilt(b, a, X1)
    return X2

def get_quality_features(ppg_data,ppg_fs=64,window_size=2.5):
    ppg_data_final = []
    n = int(ppg_fs*window_size/2)
    for i in range(n,ppg_data.shape[0]-n,1):
        tmp = []
        tmp.append(ppg_data[i,0])
        tmp.append(ppg_data[i,1])
        tmp.extend([-1]*4)
#         sample = ppg_data[(i-n):(i+n),1]
#         tmp.append(skew(sample))
#         tmp.append(kurtosis(sample))
#         tmp.append(iqr(sample))
#         f,pxx = signal.welch(sample,fs=ppg_fs,nperseg=len(sample)//2,nfft=10000,axis=0)
#         tmp.append(np.trapz(pxx[np.where((f>=.8)&(f<=2.5))[0]])/np.trapz(pxx))
        ppg_data_final.append(np.array(tmp))
    return np.array(ppg_data_final)


def save_participant_data(filename,ecg_fs = 700,ppg_fs = 64,acc_fs=32,window_size=8):
    data = pickle.load(open(filename,'rb'),encoding='latin1')
    ppg_data = data['signal']['wrist']['BVP']
    acc_data = data['signal']['wrist']['ACC']/64
    ecg_data = data['signal']['chest']['ECG']
    respiration_data = data['signal']['chest']['Resp']
    label_data = data['label']
    total_seconds = ppg_data.shape[0]/ppg_fs
    start_ts = datetime.utcnow().timestamp()
    ecg_ts = start_ts + np.arange(0,total_seconds,1/ecg_fs)
    acc_ts = start_ts + np.arange(0,total_seconds,1/acc_fs)
    
    label_data = np.concatenate([ecg_ts.reshape(-1,1),label_data.reshape(-1,1)],axis=1)
    acc_data = np.concatenate([acc_ts.reshape(-1,1),acc_data],axis=1)
    respiration_ts = ecg_ts
    respiration_data = np.vstack([respiration_ts,respiration_data.reshape(-1)]).T
    
    ecg_data = np.vstack([ecg_ts,ecg_data.reshape(-1)]).T
    ecg_rr1 = get_ecg_rr(ecg_data)
    ecg_rr = ecg_rr1.values
    
    ppg_ts = start_ts + np.arange(0,total_seconds,1/ppg_fs)
    
    ppg_data = np.vstack([ppg_ts,ppg_data.reshape(-1)]).T
    respiration_data = bandpass_filter_respiration(respiration_data,Fs=ecg_fs,fil_type='ppg')
    respiration_data[:,0] = respiration_data[:,0]*1000
    peak_index,valley_index = compute_peak_valley(respiration_data)
    peak_data = respiration_data[peak_index]
    valley_data = respiration_data[valley_index]
    rip_feature = rip_cycle_feature_computation(peak_data,valley_data)[:,:5]
    rip_feature[:,:2] = rip_feature[:,:2]/1000
    ppg_data = get_quality_features(ppg_data)
    ppg_data = pd.DataFrame(ppg_data,columns=['time','ppg','skew','kurtosis','iqr','relative_power']).dropna().sort_values('time').reset_index(drop=True)
    ppg_data['timestamp'] = ppg_data['time'].apply(lambda a:datetime.utcfromtimestamp(a))
    respiration_data[:,0] = respiration_data[:,0]/1000
    all_data = []
    for i in range(0,ppg_data.shape[0]-window_size*ppg_fs,window_size*ppg_fs//4):
        a = ppg_data.loc[i:i+window_size*ppg_fs-1]
        b = respiration_data[np.where((respiration_data[:,0]>=a['time'].min())&(respiration_data[:,0]<a['time'].max()))[0],1].reshape(-1,1)
        all_data.append([a['time'].min(),a['time'].max(),
                         a[['time','ppg','skew','kurtosis','iqr','relative_power']].sort_values('time').reset_index(drop=True),b])
    
    ppg_windows = pd.DataFrame(all_data,columns=['start_time','end_time','data','respiration'])
    ppg_windows['ecg_rr'] = ppg_windows.apply(lambda a:np.mean(ecg_rr[np.where((ecg_rr[:,0]>=a['start_time'])&(ecg_rr[:,0]<a['end_time']))[0],1]),axis=1)
    ppg_windows['inspiration_duration'] = ppg_windows.apply(lambda a:np.mean(rip_feature[np.where((rip_feature[:,1]>=a['start_time'])&(rip_feature[:,0]<a['end_time']))[0],2]),axis=1)
    ppg_windows['expiration_duration'] = ppg_windows.apply(lambda a:np.mean(rip_feature[np.where((rip_feature[:,1]>=a['start_time'])&(rip_feature[:,0]<a['end_time']))[0],3]),axis=1)
    ppg_windows['respiration_duration'] = ppg_windows.apply(lambda a:np.mean(rip_feature[np.where((rip_feature[:,1]>=a['start_time'])&(rip_feature[:,0]<a['end_time']))[0],4]),axis=1)
    ppg_windows['acc_window'] = ppg_windows.apply(lambda a: acc_data[np.where((acc_data[:,0]>=a['start_time'])&(acc_data[:,0]<a['end_time']))[0],:],axis=1)
    ppg_windows['label'] = ppg_windows.apply(lambda a: mode(label_data[np.where((label_data[:,0]>=a['start_time'])&(label_data[:,0]<a['end_time']))[0],1])[0][0],axis=1)
    
    ppg_windows = ppg_windows.sort_values('start_time').reset_index(drop=True)
    hr = heart_rate()
    ppg_rr_col = []
    for i,a in ppg_windows.iterrows():
        ppg_rr_col.append(hr.get_rr_value(a['data'].sort_values('time').reset_index(drop=True)['ppg'].values,
                                        a['acc_window'][a['acc_window'][:,0].argsort(),1:],
                                        a['ecg_rr']*1000))
        
        
    ppg_windows['ppg_rr'] = np.array(ppg_rr_col)/1000
    if not os.path.isdir(output_directory+str(window_size)):
        os.makedirs(output_directory+str(window_size))
    final_path = output_directory+str(window_size)+'/'
    participant_name = filename.split('/')[-1]
    pickle.dump(ppg_windows,open(final_path+participant_name,'wb'))
    return []

from joblib import Parallel,delayed
output_directory = '../data/'
final = Parallel(n_jobs=20,verbose=2)(delayed(save_participant_data)(f,window_size=8) for f in filelists)
# output = [save_participant_data(f,window_size=8) for f in filelists[:1]]

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   8 out of  15 | elapsed:  3.8min remaining:  3.3min
[Parallel(n_jobs=20)]: Done  15 out of  15 | elapsed:  5.0min finished


In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mode
filepath = '../data/8/'
filelists = [filepath+a for a in os.listdir(filepath) if a[-1] not in ['s','f']]
from datetime import datetime
y_stress = []
y_participant = []
X_hr = []
X_time = []
import pickle
for i,f in enumerate(filelists):
    data = pickle.load(open(f,'rb'))
#     data['ppg_rr'] = (data['ppg_rr'] - data['ppg_rr'].mean())/data['ppg_rr'].std()
    data = data.sort_values('start_time').reset_index(drop=True)
    data['timestamp'] = data['start_time'].apply(lambda a:datetime.utcfromtimestamp(a))
    for j in range(0,data.shape[0],1):
        temp = data[j:(j+30)]
        if temp.shape[0]!=30:
            continue
        labels = temp['label'].values
        y_stress.append(mode(labels)[0][0])
        X_hr.append(temp['ppg_rr'].values.reshape(1,30))
        y_participant.append(i)
        X_time.append(temp['start_time'].values[0])

y_stress = np.array(y_stress)
y_participant = np.array(y_participant)
X_time = np.array(X_time)
X_hr = np.concatenate(X_hr)
print(X_time.shape,y_stress.shape,y_participant.shape,X_hr.shape)

import pickle
pickle.dump([X_hr,y_stress,y_participant,X_time],open('../data/tabular_data_60_seconds_ppg_rr.p','wb'))

(42928,) (42928,) (42928,) (42928, 30)


In [34]:
import os
import numpy as np
import pandas as pd

filepath = '../data/8/'
filelists = [filepath+a for a in os.listdir(filepath) if a[-1] not in ['s','f']]

from datetime import datetime
X_ppg = []
X_acl = []
y = []
y_participant = []
X_hr_windows = []
X_time = []
import pickle
for i,f in enumerate(filelists):
    data = pickle.load(open(f,'rb'))
    data = data.sort_values('start_time').reset_index(drop=True)
    X_time.extend(list(data['start_time'].values))
    X_ppg.extend([a.values[:,1:3].reshape(1,-1,2) for a in data['data'].values])
    X_acl.extend([a[:,1:].reshape(1,-1,6) for a in data['acc_window'].values])
    y.extend(list(data['ecg_rr'].values))
    y_participant.extend([i]*data.shape[0])
    data['participant'] = i
    data['timestamp'] = data['start_time'].apply(lambda a:datetime.utcfromtimestamp(a))
#     data = data.sort_values('timestamp').reset_index(drop=True)
#     hr_windows = [df[['ecg_rr','participant']].values for i,df in 
#                   data.groupby(pd.Grouper(key='timestamp',freq='20S')) if df.shape[0]==10]
#     X_hr_windows.extend(hr_windows)

X_acl = np.concatenate(X_acl)
X_ppg = np.concatenate(X_ppg)
y = np.array(y)
y_participant = np.array(y_participant)
X_time = np.array(X_time)
print(X_time.shape,X_acl.shape,X_ppg.shape,y.shape,y_participant.shape)

import pickle
pickle.dump([X_acl,X_ppg,y,y_participant,X_time],open('../data/tabular_data.p','wb'))